##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/custom_callback"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# Keras custom callbacks
A custom callback is a powerful tool to customize the behavior of a Keras model during training, evaluation, or inference, including reading/changing the Keras model. Examples include `tf.keras.callbacks.TensorBoard` where the training progress and results can be exported and visualized with TensorBoard, or `tf.keras.callbacks.ModelCheckpoint` where the model is automatically saved during training, and more. In this guide, you will learn what Keras callback is, when it will be called, what it can do, and how you can build your own. Towards the end of this guide, there will be demos of creating a couple of simple callback applications to get you started on your custom callback.

## Setup

In [2]:
import tensorflow as tf

C:\Users\Melody\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Introduction to Keras callbacks
In Keras, `Callback` is a python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. You can pass a list of callbacks (as the keyword argument `callbacks`) to any of `tf.keras.Model.fit()`, `tf.keras.Model.evaluate()`, and `tf.keras.Model.predict()` methods. The methods of the callbacks will then be called at different stages of training/evaluating/inference.

To get started, let's import tensorflow and define a simple Sequential Keras model:

In [3]:
# Define the Keras model to add callbacks to
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(1, activation = 'linear', input_dim = 784))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1), loss='mean_squared_error', metrics=['mae'])
    return model

Then, load the MNIST data for training and testing from Keras datasets API:

In [4]:
# Load example MNIST data and pre-process it
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

Now, define a simple custom callback to track the start and end of every batch of data. During those calls, it prints the index of the current batch.

In [5]:
import datetime

class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_train_batch_begin(self, batch, logs=None):
        print('Training: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

    def on_train_batch_end(self, batch, logs=None):
        print('Training: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

    def on_test_batch_begin(self, batch, logs=None):
        print('Evaluating: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

    def on_test_batch_end(self, batch, logs=None):
        print('Evaluating: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

Providing a callback to model methods such as `tf.keras.Model.fit()` ensures the methods are called at those stages:

In [9]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          epochs=1,
          steps_per_epoch=5,
          verbose=0,
          callbacks=[MyCustomCallback()])

Training: batch 0 begins at 10:18:27.570186
Training: batch 0 ends at 10:18:27.750879
Training: batch 1 begins at 10:18:27.750879
Training: batch 1 ends at 10:18:27.800540
Training: batch 2 begins at 10:18:27.800540
Training: batch 2 ends at 10:18:27.849666
Training: batch 3 begins at 10:18:27.849666
Training: batch 3 ends at 10:18:27.895737
Training: batch 4 begins at 10:18:27.896739
Training: batch 4 ends at 10:18:27.943075
Training: batch 0 begins at 10:18:27.943075
Training: batch 0 ends at 10:18:27.989461
Training: batch 1 begins at 10:18:27.993606
Training: batch 1 ends at 10:18:28.043752
Training: batch 2 begins at 10:18:28.043752
Training: batch 2 ends at 10:18:28.090689
Training: batch 3 begins at 10:18:28.094774
Training: batch 3 ends at 10:18:28.142494
Training: batch 4 begins at 10:18:28.142494
Training: batch 4 ends at 10:18:28.187691


## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
#### [`fit()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
#### [`evaluate()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
#### [`predict()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.


In [10]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=5,
          callbacks=[MyCustomCallback()])

Evaluating: batch 0 begins at 10:20:29.010647
Evaluating: batch 0 ends at 10:20:29.049856
Evaluating: batch 1 begins at 10:20:29.049856
Evaluating: batch 1 ends at 10:20:29.055550
Evaluating: batch 2 begins at 10:20:29.055550
Evaluating: batch 2 ends at 10:20:29.058150
Evaluating: batch 3 begins at 10:20:29.058150
Evaluating: batch 3 ends at 10:20:29.063496
Evaluating: batch 4 begins at 10:20:29.063496
Evaluating: batch 4 ends at 10:20:29.070890


## An overview of callback methods


### Common methods for training/testing/predicting
For training, testing, and predicting, following methods are provided to be overridden.
#### `on_(train|test|predict)_begin(self, logs=None)`
Called at the beginning of `fit`/`evaluate`/`predict`.
#### `on_(train|test|predict)_end(self, logs=None)`
Called at the end of `fit`/`evaluate`/`predict`.
#### `on_(train|test|predict)_batch_begin(self, batch, logs=None)`
Called right before processing a batch during training/testing/predicting. Within this method, `logs` is a dict with `batch` and `size` available keys, representing the current batch number and the size of the batch.
#### `on_(train|test|predict)_batch_end(self, batch, logs=None)`
Called at the end of training/testing/predicting a batch. Within this method, `logs` is a dict containing the stateful metrics result.

### Training specific methods
In addition, for training, following are provided.
#### on_epoch_begin(self, epoch, logs=None)
Called at the beginning of an epoch during training.
#### on_epoch_end(self, epoch, logs=None)
Called at the end of an epoch during training.


### Usage of `logs` dict
The `logs` dict contains the loss value, and all the metrics at the end of a batch or epoch. Example includes the loss and mean absolute error.

In [15]:
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):

    def on_train_batch_end(self, batch, logs=None):
        print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

    def on_test_batch_end(self, batch, logs=None):
        print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

    def on_epoch_end(self, epoch, logs=None):
        print('The average loss for epoch {} is {:7.2f} and mean absolute error is {:7.2f}.'.format(epoch, logs['loss'], logs['mean_absolute_error']))
        print(logs)
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=3,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback()])

For batch 0, loss is   32.72.
For batch 1, loss is  894.47.
For batch 2, loss is   28.78.
For batch 3, loss is    9.19.
For batch 4, loss is    7.37.
The average loss for epoch 0 is  194.51 and mean absolute error is    8.41.
{'loss': 194.5066930770874, 'mean_absolute_error': 8.407329}
For batch 0, loss is    6.50.
For batch 1, loss is    5.92.
For batch 2, loss is    5.51.
For batch 3, loss is    5.23.
For batch 4, loss is    5.02.
The average loss for epoch 1 is    5.64 and mean absolute error is    1.99.
{'loss': 5.635849094390869, 'mean_absolute_error': 1.9933155}
For batch 0, loss is    4.86.
For batch 1, loss is    4.74.
For batch 2, loss is    4.64.
For batch 3, loss is    4.56.
For batch 4, loss is    4.48.
The average loss for epoch 2 is    4.66 and mean absolute error is    1.77.
{'loss': 4.657289314270019, 'mean_absolute_error': 1.7684387}


Similarly, one can provide callbacks in `evaluate()` calls.

In [16]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=20,
          callbacks=[LossAndErrorPrintingCallback()])

For batch 0, loss is    4.20.
For batch 1, loss is    4.20.
For batch 2, loss is    4.20.
For batch 3, loss is    4.20.
For batch 4, loss is    4.20.
For batch 5, loss is    4.20.
For batch 6, loss is    4.20.
For batch 7, loss is    4.20.
For batch 8, loss is    4.20.
For batch 9, loss is    4.20.
For batch 10, loss is    4.20.
For batch 11, loss is    4.20.
For batch 12, loss is    4.20.
For batch 13, loss is    4.20.
For batch 14, loss is    4.20.
For batch 15, loss is    4.20.
For batch 16, loss is    4.20.
For batch 17, loss is    4.20.
For batch 18, loss is    4.20.
For batch 19, loss is    4.20.


## Examples of Keras callback applications
The following section will guide you through creating simple Callback applications.

### Early stopping at minimum loss
First example showcases the creation of a `Callback` that stops the Keras training when the minimum of loss has been reached by mutating the attribute `model.stop_training` (boolean). Optionally, the user can provide an argument `patience` to specify how many epochs the training should wait before it eventually stops.

`tf.keras.callbacks.EarlyStopping` provides a more complete and general implementation.

In [18]:
import numpy as np

class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
  """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

  def __init__(self, patience=0):
    super(EarlyStoppingAtMinLoss, self).__init__()

    self.patience = patience

    # best_weights to store the weights at which the minimum loss occurs.
    self.best_weights = None

  def on_train_begin(self, logs=None):
    # The number of epoch it has waited when loss is no longer minimum.
    self.wait = 0
    # The epoch the training stops at.
    self.stopped_epoch = 0
    # Initialize the best as infinity.
    self.best = np.Inf

  def on_epoch_end(self, epoch, logs=None):
    current = logs.get('loss')
    if np.less(current, self.best):
      self.best = current
      self.wait = 0
      # Record the best weights if current results is better (less).
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if self.wait >= self.patience:
        self.stopped_epoch = epoch
        self.model.stop_training = True
        print('Restoring model weights from the end of the best epoch.')
        self.model.set_weights(self.best_weights)

  def on_train_end(self, logs=None):
    if self.stopped_epoch > 0:
      print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

In [20]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=15,
          epochs=30,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()])

For batch 0, loss is   28.11.
For batch 1, loss is  921.95.
For batch 2, loss is   25.25.
For batch 3, loss is    8.76.
For batch 4, loss is    7.18.
For batch 5, loss is    6.38.
For batch 6, loss is    5.83.
For batch 7, loss is    5.44.
For batch 8, loss is    5.17.
For batch 9, loss is    4.97.
For batch 10, loss is    4.82.
For batch 11, loss is    4.71.
For batch 12, loss is    4.61.
For batch 13, loss is    4.54.
For batch 14, loss is    4.47.
The average loss for epoch 0 is   69.48 and mean absolute error is    4.01.
{'loss': 69.48049233754476, 'mean_absolute_error': 4.0090423}
For batch 0, loss is    4.40.
For batch 1, loss is    4.35.
For batch 2, loss is    4.30.
For batch 3, loss is    4.25.
For batch 4, loss is    4.20.
For batch 5, loss is    4.16.
For batch 6, loss is    4.12.
For batch 7, loss is    4.10.
For batch 8, loss is    4.10.
For batch 9, loss is    4.34.
For batch 10, loss is    6.39.
For batch 11, loss is   23.98.
For batch 12, loss is  140.86.
For batch 13, 

### Learning rate scheduling

One thing that is commonly done in model training is changing the learning rate as more epochs have passed. Keras backend exposes `get_value` API which can be used to set the variables. In this example, we're showing how a custom Callback can be used to dynamically change the learning rate.

Note: This is just an example implementation. See `callbacks.LearningRateScheduler` and `keras.optimizers.schedules` for more general implementations.

In [21]:
class LearningRateScheduler(tf.keras.callbacks.Callback):
  """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

  def __init__(self, schedule):
    super(LearningRateScheduler, self).__init__()
    self.schedule = schedule

  def on_epoch_begin(self, epoch, logs=None):
    if not hasattr(self.model.optimizer, 'lr'):
      raise ValueError('Optimizer must have a "lr" attribute.')
    # Get the current learning rate from model's optimizer.
    lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
    # Call schedule function to get the scheduled learning rate.
    scheduled_lr = self.schedule(epoch, lr)
    # Set the value back to the optimizer before this epoch starts
    tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
    print('\nEpoch %05d: Learning rate is %6.4f.' % (epoch, scheduled_lr))

In [22]:
LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05), (6, 0.01), (9, 0.005), (12, 0.001)
]

def lr_schedule(epoch, lr):
  """Helper function to retrieve the scheduled learning rate based on epoch."""
  if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
    return lr
  for i in range(len(LR_SCHEDULE)):
    if epoch == LR_SCHEDULE[i][0]:
      return LR_SCHEDULE[i][1]
  return lr

model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=15,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), LearningRateScheduler(lr_schedule)])


Epoch 00000: Learning rate is 0.1000.
For batch 0, loss is   29.63.
For batch 1, loss is  917.48.
For batch 2, loss is   26.52.
For batch 3, loss is    9.24.
For batch 4, loss is    7.48.
The average loss for epoch 0 is  198.07 and mean absolute error is    8.37.
{'loss': 198.07281913757325, 'mean_absolute_error': 8.371842}

Epoch 00001: Learning rate is 0.1000.
For batch 0, loss is    6.57.
For batch 1, loss is    5.95.
For batch 2, loss is    5.53.
For batch 3, loss is    5.23.
For batch 4, loss is    5.02.
The average loss for epoch 1 is    5.66 and mean absolute error is    1.99.
{'loss': 5.659059619903564, 'mean_absolute_error': 1.9914137}

Epoch 00002: Learning rate is 0.1000.
For batch 0, loss is    4.86.
For batch 1, loss is    4.74.
For batch 2, loss is    4.64.
For batch 3, loss is    4.56.
For batch 4, loss is    4.49.
The average loss for epoch 2 is    4.66 and mean absolute error is    1.77.
{'loss': 4.658311939239502, 'mean_absolute_error': 1.7664484}

Epoch 00003: Learn

### Standard Keras callbacks
Be sure to check out the existing Keras callbacks by [visiting the API doc](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks). Applications include logging to CSV, saving the model, visualizing on TensorBoard and a lot more.

Here, stateful_metrics are the name of the metrics that you don’t want to average over an epoch. All the metrics names should be passed in the form of iterable like lists etc. For instance, stateful_metrics=[‘acc’,’loss’].

In [ ]:
keras.callbacks.BaseLogger(stateful_metrics=None)